Note: this still isn't working as the tickers in the constituents don't match tickers existing now. Maybe work on getting data using the isin from the ishares website? Maybe manually create symbol change table

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

data = requests.get("https://www.ishares.com/us/products/239726/#tabsAll").content
soup = BeautifulSoup(data)

holdings = soup.find("div", {"id":"holdings"})
date_list_div = holdings.find_all("div","component-date-list")[1] # second one has all dates
date_list_div.find_all("option")

date_list = [option.attrs["value"] for option in date_list_div.find_all("option")]



In [30]:
import json
from datetime import datetime
constituents = pd.Series()

for date_str in date_list:
    data = requests.get(f"https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?tab=all&fileType=json&asOfDate={date_str}").content[3:]
    tickers = json.loads(data)
    tickers = [(arr[0],arr[1]) for arr in tickers['aaData']]
    date = datetime.strptime(date_str, "%Y%m%d")
    constituents[date] = tickers

In [31]:
constituents

2019-05-09    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2019-04-30    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2019-03-29    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2019-02-28    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2019-01-31    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2018-12-31    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2018-11-30    [(MSFT, MICROSOFT CORP), (AAPL, APPLE INC), (A...
2018-10-31    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-09-28    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-08-31    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-07-31    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-06-29    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-05-31    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-04-30    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-03-29    [(AAPL, APPLE INC), (MSFT, MICROSOFT CORP), (A...
2018-02-28    [(AAPL, APPLE INC), (MSFT,

In [32]:
constituents.to_csv("constituents.csv")

In [33]:
import pandas as pd
import ast
c = pd.read_csv('constituents.csv',header=None, index_col=0, squeeze=True, parse_dates=True, converters={1:ast.literal_eval})

In [34]:
c = c.iloc[::-1] # reverse into cronlogical order

In [35]:
from datetime import datetime, timedelta

In [36]:
c.index[0]

Timestamp('2006-09-29 00:00:00')

In [5]:
c.index[1].date()

datetime.date(2006, 10, 31)

In [38]:
c[0][0]

('PMCS', 'PMC-SIERRA INC.')

In [6]:
df = pd.read_csv("survivor-free/WIKI_PRICES.csv", parse_dates=True)

In [7]:
dfs = dict(tuple(df.groupby('ticker')))
for ticker in dfs:
    dfs[ticker].set_index("date", inplace=True)


In [8]:
dfs["AAPL"].tail()

,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
date,,,,,,,,,,,,,
2018-03-21,AAPL,175.04,175.09,171.26,171.270,35247358.0,0.0,1.0,175.04,175.09,171.26,171.270,35247358.0
2018-03-22,AAPL,170.00,172.68,168.60,168.845,41051076.0,0.0,1.0,170.00,172.68,168.60,168.845,41051076.0
2018-03-23,AAPL,168.39,169.92,164.94,164.940,40248954.0,0.0,1.0,168.39,169.92,164.94,164.940,40248954.0
2018-03-26,AAPL,168.07,173.10,166.44,172.770,36272617.0,0.0,1.0,168.07,173.10,166.44,172.770,36272617.0
2018-03-27,AAPL,173.68,175.15,166.92,168.340,38962839.0,0.0,1.0,173.68,175.15,166.92,168.340,38962839.0


In [39]:
dfs["A"].loc[str(c.index[0].date()):str(c.index[1].date())]

,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
date,,,,,,,,,,,,,
2006-09-29,A,33.00,33.17,32.64,32.69,1455800.0,0.0,1.0,22.513943,22.629924,22.268336,22.302448,1455800.0
2006-10-02,A,32.50,32.78,32.12,32.32,2832000.0,0.0,1.0,22.172822,22.363850,21.913571,22.050019,2832000.0
2006-10-03,A,32.07,32.58,32.05,32.24,2303100.0,0.0,1.0,21.879459,22.227402,21.865814,21.995440,2303100.0
2006-10-04,A,32.15,33.12,32.14,33.04,2533100.0,0.0,1.0,21.934038,22.595812,21.927216,22.541232,2533100.0
2006-10-05,A,32.84,33.01,32.50,32.90,1699400.0,0.0,1.0,22.404784,22.520765,22.172822,22.445719,1699400.0
2006-10-06,A,32.65,33.40,32.42,33.29,2098700.0,0.0,1.0,22.275158,22.786839,22.118243,22.711792,2098700.0
2006-10-09,A,33.05,33.58,32.77,33.42,2789000.0,0.0,1.0,22.548055,22.909642,22.357027,22.800484,2789000.0
2006-10-10,A,33.35,33.61,33.22,33.35,2049800.0,0.0,1.0,22.752727,22.930109,22.664036,22.752727,2049800.0
2006-10-11,A,33.00,33.66,32.79,33.18,2566500.0,0.0,1.0,22.513943,22.964221,22.370672,22.636746,2566500.0


In [10]:
c.index[-20:]

DatetimeIndex(['2017-10-31', '2017-11-30', '2017-12-29', '2018-01-31',
               '2018-02-28', '2018-03-29', '2018-04-30', '2018-05-31',
               '2018-06-29', '2018-07-31', '2018-08-31', '2018-09-28',
               '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31',
               '2019-02-28', '2019-03-29', '2019-04-30', '2019-05-09'],
              dtype='datetime64[ns]', name=0, freq=None)

In [57]:
constituents = c['2013-02-28':'2018-02-28']

In [64]:
constituents

0
2013-02-28    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-03-28    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-04-30    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-05-31    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-06-28    [(XOM, EXXON MOBIL CORP), (AAPL*, APPLE INC), ...
2013-07-31    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-08-30    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-09-30    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-10-31    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-11-29    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2013-12-31    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2014-01-31    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2014-02-28    [(AAPL*, APPLE INC), (XOM, EXXON MOBIL CORP), ...
2014-03-31    [(AAPL, APPLE INC), (XOM, EXXON MOBIL CORP), (...
2014-04-30    [(AAPL, APPLE INC), (XOM, EXXON MOBIL CORP), (...
2014-05-30    [(AAPL, APPLE INC), (XOM

In [68]:
data = {}
fails = set()

import re
import pandas_datareader.data as web


for i in range(0, len(constituents) - 1):
    start = constituents.index[i].date()
    end = (constituents.index[i + 1].to_pydatetime() - timedelta(days=1)).date()
    print(f"{start} - {end}")
    for company in constituents[i]:
        if company[0] not in dfs:
            cleaned = re.sub(r'[^A-Z]+', '', company[0])
            if cleaned not in dfs:
                fails.add(company)

2013-02-28 - 2013-03-27
2013-03-28 - 2013-04-29
2013-04-30 - 2013-05-30
2013-05-31 - 2013-06-27
2013-06-28 - 2013-07-30
2013-07-31 - 2013-08-29
2013-08-30 - 2013-09-29
2013-09-30 - 2013-10-30
2013-10-31 - 2013-11-28
2013-11-29 - 2013-12-30
2013-12-31 - 2014-01-30
2014-01-31 - 2014-02-27
2014-02-28 - 2014-03-30
2014-03-31 - 2014-04-29
2014-04-30 - 2014-05-29
2014-05-30 - 2014-06-29
2014-06-30 - 2014-07-30
2014-07-31 - 2014-08-28
2014-08-29 - 2014-09-29
2014-09-30 - 2014-10-30
2014-10-31 - 2014-11-27
2014-11-28 - 2014-12-30
2014-12-31 - 2015-01-29
2015-01-30 - 2015-02-26
2015-02-27 - 2015-03-30
2015-03-31 - 2015-04-29
2015-04-30 - 2015-05-28
2015-05-29 - 2015-06-29
2015-06-30 - 2015-07-30
2015-07-31 - 2015-08-30
2015-08-31 - 2015-09-29
2015-09-30 - 2015-10-29
2015-10-30 - 2015-11-29
2015-11-30 - 2015-12-30
2015-12-31 - 2016-01-28
2016-01-29 - 2016-02-28
2016-02-29 - 2016-03-30
2016-03-31 - 2016-04-28
2016-04-29 - 2016-05-30
2016-05-31 - 2016-06-29
2016-06-30 - 2016-07-28
2016-07-29 - 201

In [71]:
fails

{('-', 'BLACKROCK FDS III'),
 ('4XS', 'EXPRESS SCRIPTS HOLDING'),
 ('8686', 'AFLAC INC.'),
 ('AABA', 'ALTABA INC'),
 ('AAZ', 'ANADARKO PETROLEUM CORP'),
 ('ACT', 'ACTAVIS INC.'),
 ('ACT', 'ACTAVIS PLC'),
 ('AG4', 'ALLERGAN'),
 ('ALD', 'HONEYWELL INTERNATIONAL INC'),
 ('BF.B', 'BROWN FORMAN CORP CLASS B'),
 ('BF/B', 'BROWN FORMAN CORP CLASS B'),
 ('BFB', 'BROWN FORMAN CORP CLASS B'),
 ('BLD WI', 'TOPBUILD CORP WHEN ISSUED'),
 ('BLKFDS', 'BLK CSH FND TREASURY SL AGENCY'),
 ('BMC', 'BMC SOFTWARE INC.'),
 ('BRKB', 'BERKSHIRE HATHAWAY INC CLASS B'),
 ('BTI', 'BRITISH AMERICAN TOBACCO ADR REPRE'),
 ('CC WI', 'CHEMOURS ORD WHEN ISSUED'),
 ('CRC', 'CALIFORNIA RESOURCES CORP'),
 ('CVH', 'COVENTRY HEALTH CARE INC'),
 ('CWGL', 'CRIMSON WINE GROUP LTD'),
 ('DC7', 'DISCOVER FINANCIAL SERVICES'),
 ('ESH5', 'S&P500 EMINI MAR 15'),
 ('ESH6', 'S&P500 EMINI MAR 16'),
 ('ESH7', 'S&P500 EMINI MAR 17'),
 ('ESH8', 'S&P500 EMINI MAR 18'),
 ('ESM5', 'S&P500 EMINI JUN 15'),
 ('ESM6', 'S&P500 EMINI JUN 16'),
 (

In [72]:
not_found = set()
for fail in fails:
    try:
        test = web.DataReader(fail[0], "yahoo")
    except:
        not_found.add(fail)

In [75]:
not_found

{('-', 'BLACKROCK FDS III'),
 ('4XS', 'EXPRESS SCRIPTS HOLDING'),
 ('8686', 'AFLAC INC.'),
 ('AAZ', 'ANADARKO PETROLEUM CORP'),
 ('AG4', 'ALLERGAN'),
 ('BF.B', 'BROWN FORMAN CORP CLASS B'),
 ('BF/B', 'BROWN FORMAN CORP CLASS B'),
 ('BLD WI', 'TOPBUILD CORP WHEN ISSUED'),
 ('BLKFDS', 'BLK CSH FND TREASURY SL AGENCY'),
 ('CC WI', 'CHEMOURS ORD WHEN ISSUED'),
 ('DC7', 'DISCOVER FINANCIAL SERVICES'),
 ('ESH5', 'S&P500 EMINI MAR 15'),
 ('ESH6', 'S&P500 EMINI MAR 16'),
 ('ESH7', 'S&P500 EMINI MAR 17'),
 ('ESH8', 'S&P500 EMINI MAR 18'),
 ('ESM5', 'S&P500 EMINI JUN 15'),
 ('ESM6', 'S&P500 EMINI JUN 16'),
 ('ESM7', 'S&P500 EMINI JUN 17'),
 ('ESU5', 'S&P500 EMINI SEP 15'),
 ('ESU6', 'S&P500 EMINI SEP 16'),
 ('ESU7', 'S&P500 EMINI SEP 17'),
 ('ESZ4', 'S&P500 EMINI DEC 14'),
 ('ESZ5', 'S&P500 EMINI DEC 15'),
 ('ESZ6', 'S&P500 EMINI DEC 16'),
 ('ESZ7', 'S&P500 EMINI DEC 17'),
 ('GGQ7', 'GOOGLE INC CLASS C'),
 ('HAWKB', 'BLACKHAWK NETWORK HOLDINGS INC CLA'),
 ('HNZ', 'H.J. HEINZ CO.'),
 ('LOM', 'LOC

In [133]:
import time

In [134]:
def clean_ticker(ticker):
    rename_table = {
        "-": "LPRAX", # BlackRock LifePath Dynamic Retirement Fund
        "8686": "AFL", # AFLAC
        "4XS": "ESRX", # Express Scripts Holding Company 
        "AAZ": "APC", # Anadarko Petroleum Corporation
        "AG4": "AGN", # Allergan plc
        "BFB": "BF_B", # Brown-Forman Corporation
        "BF.B": "BF_B", # Brown-Forman Corporation
        "BF/B": "BF_B", # Brown-Forman Corporation
        "BLD WI": "BLD", # TopBuild Corp.
        "BRKB": "BRK_B", # Berkshire Hathaway Inc.
        "CC WI": "CC", # The Chemours Company
        "DC7": "DFS", # Discover Financial Services
        "GGQ7": "GOOG", # Alphabet Inc. Class C
        "HNZ": "KHC", # The Kraft Heinz Company
        "LOM": "LMT", # Lockheed Martin Corp.
        "LTD": "LB", # L Brands Inc.
        "LTR": "L", # Loews Corporation
        "MPN": "MPC", # Marathon Petroleum Corp.
        "MWZ": "MET", # Metlife Inc.
        "MX4A": "CME", # CME Group Inc.
        "NCRA": "NWSA", # News Corporation
        "NTH": "NOC", # Northrop Grumman Crop.
        "PA9": "TRV", # The Travelers Companies, Inc.
        "QCI": "QCOM", # Qualcomm Inc.
        "RN7": "RF", # Regions Financial Corp
        "SLBA": "SLB", # Schlumberger Limited
        "SYF-W": "SYF", # Synchrony Financial
        "SWG": "SCHW", # The Charles Schwab Corporation 
        "UAC/C": "UAA", # Under Armour Inc Class A
        "UBSFT": "UBSFY", # Ubisoft Entertainment
        "USX1": "X", # United States Steel Corporation
        "UUM": "UNM", # Unum Group
        "VISA": "V", # Visa Inc 
    }
    if ticker in rename_table:
        clean = rename_table[ticker]
    else:
        clean = re.sub(r'[^A-Z]+', '', ticker)
    return clean
    

def quandl_data(ticker, start, end):
    if ticker in dfs:
        df = dfs[ticker][start:end]
    else:
        clean = clean_ticker(ticker)
        if clean in dfs:
            df = dfs[clean][start:end]
        else:
            return None
    df = df.drop(['open','high','low','close', 'volume','ex-dividend','split_ratio', 'ticker'], axis=1)
    df = df.rename(index=str, columns={"adj_open": "open",
                                       "adj_high": "high",
                                       "adj_low": "low",
                                       "adj_close": "close",
                                       "adj_volume": "volume"})
    return df

def yahoo_data(ticker, start, end):
    clean = clean_ticker(ticker)
    try:
        df = web.DataReader(clean, "yahoo", start, end)
        
    except:
        time.sleep(1)
        try:
            df = web.DataReader(clean, "yahoo", start, end)
        except:
            return None
    adjfactor = df["Close"] / df["Adj Close"]
    df["Open"] /= adjfactor
    df["High"] /= adjfactor
    df["Low"] /= adjfactor
    df["Close"] = df["Adj Close"]
    df["Volume"] *= adjfactor
    df = df.drop(["Adj Close"], axis=1)
    df = df.rename(str.lower, axis='columns')
    return df

In [136]:
data = {}
fails = set()

for i in range(0, len(constituents) - 1):
    start = str(constituents.index[i].date())
    end = str((constituents.index[i + 1].to_pydatetime() - timedelta(days=1)).date())
    for company in constituents[i]:
        if company in fails:
            continue
        df = quandl_data(company[0], start, end)
        if df is None:
            df = yahoo_data(company[0], start, end)
        if df is None:
            fails.add(company)
            continue
        if company[0] in data:
            data[company[0]].append(df)
        else:
            data[company[0]] = df

In [137]:
fails

{('ACT', 'ACTAVIS INC.'),
 ('ACT', 'ACTAVIS PLC'),
 ('BLKFDS', 'BLK CSH FND TREASURY SL AGENCY'),
 ('BMC', 'BMC SOFTWARE INC.'),
 ('CVH', 'COVENTRY HEALTH CARE INC'),
 ('ESH5', 'S&P500 EMINI MAR 15'),
 ('ESH6', 'S&P500 EMINI MAR 16'),
 ('ESH7', 'S&P500 EMINI MAR 17'),
 ('ESH8', 'S&P500 EMINI MAR 18'),
 ('ESM5', 'S&P500 EMINI JUN 15'),
 ('ESM6', 'S&P500 EMINI JUN 16'),
 ('ESM7', 'S&P500 EMINI JUN 17'),
 ('ESU5', 'S&P500 EMINI SEP 15'),
 ('ESU6', 'S&P500 EMINI SEP 16'),
 ('ESU7', 'S&P500 EMINI SEP 17'),
 ('ESZ4', 'S&P500 EMINI DEC 14'),
 ('ESZ5', 'S&P500 EMINI DEC 15'),
 ('ESZ6', 'S&P500 EMINI DEC 16'),
 ('ESZ7', 'S&P500 EMINI DEC 17'),
 ('HAWKB', 'BLACKHAWK NETWORK HOLDINGS INC CLA'),
 ('MARGIN_USD', 'FUTURES USD MARGIN BALANCE'),
 ('MOLX', 'MOLEX INC.'),
 ('NYX', 'NYSE EURONEXT'),
 ('PCS', 'METROPCS COMMUNICATIONS INC.'),
 ('UBFUT', 'CASH COLLATERAL USD UBFUT')}

In [141]:
for ticker, df in data.items():
    df.to_csv(f"survivor-free/{clean_ticker(ticker)}.csv")